In [2]:
%load_ext autoreload
%autoreload 2
from datasets import DatasetDict
from pysentimiento.lince.pos import load_datasets, id2label


train_dataset, dev_dataset, test_dataset = load_datasets("es")

lince_pos = DatasetDict(
    train=train_dataset,
    validation=dev_dataset,
    test=test_dataset,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Reusing dataset lince (/home/jmperez/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-f8462a766744bd0d.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-b2415cc41a42ddca.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-302bcedfefea246f.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-986aab105c31f86f.arrow
Loading cached processed dataset at /home/jmperez/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-c56998e93fa6679b.arrow
Loading cached processed 

In [3]:
id2label

['VERB',
 'PUNCT',
 'PRON',
 'NOUN',
 'DET',
 'ADV',
 'ADP',
 'INTJ',
 'CONJ',
 'ADJ',
 'AUX',
 'SCONJ',
 'PART',
 'PROPN',
 'NUM',
 'UNK',
 'X']

In [4]:
for ex in lince_pos["train"]:
    assert all(x is not None for x in ex["labels"])

¿Puede pasar que haya etiquetas que no estén en dev o test?

In [5]:
train_tags = set(tag for ex in lince_pos["train"] for tag in ex["pos"])
dev_tags = set(tag for ex in lince_pos["validation"] for tag in ex["pos"])

train_tags == dev_tags

True

In [8]:
lince_pos

DatasetDict({
    train: Dataset({
        features: ['idx', 'words', 'lid', 'pos', 'labels'],
        num_rows: 27893
    })
    validation: Dataset({
        features: ['idx', 'words', 'lid', 'pos', 'labels'],
        num_rows: 4298
    })
    test: Dataset({
        features: ['idx', 'words', 'lid', 'pos', 'labels'],
        num_rows: 10720
    })
})

In [18]:
from pysentimiento.training import load_model
from transformers import AutoModelForTokenClassification
from pysentimiento.lince.pos import label2id, id2label, tokenize_and_align_labels

base_model = "pysentimiento/robertuito-base-uncased"

max_length = 128
model, tokenizer = load_model(
    base_model, label2id=label2id, id2label=id2label,
    max_length=max_length,
    auto_class=AutoModelForTokenClassification
)

tokenized_datasets = lince_pos.map(
    lambda x: tokenize_and_align_labels(x, tokenizer),
    batched=True,
    #remove_columns=lince_ner["train"].column_names,
)



Loading model pysentimiento/robertuito-base-uncased


Some weights of the model checkpoint at pysentimiento/robertuito-base-uncased were not used when initializing RobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at pysentimiento/robertuito-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a dow

  0%|          | 0/28 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

Sanity check

In [19]:
from tqdm.auto import tqdm

problematic = []

for split in tokenized_datasets:
    for idx, ex in tqdm(enumerate(tokenized_datasets[split]), total=len(tokenized_datasets[split])):
        if not (len(ex["words"]) == (ex["word_ids"][-2] + 1)):
            problematic.append((split, idx))

print(f"{len(problematic)} problematic examples")


  0%|          | 0/27893 [00:00<?, ?it/s]

  0%|          | 0/4298 [00:00<?, ?it/s]

  0%|          | 0/10720 [00:00<?, ?it/s]

1 problematic examples


In [22]:
len(lince_pos["train"][10310]["words"])

138

Ok, es un problema del largo del tweet (qué ganas de romper las bolas que tienen, 138 palabras!)

In [23]:
assert model.config.num_labels == len(id2label)

In [24]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)



In [29]:
from transformers import TrainingArguments, Trainer
from pysentimiento.lince.pos import compute_metrics

batch_size = 32
epochs = 5

training_args = TrainingArguments(
    output_dir="/tmp/pysentimiento-prueba",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    do_eval=False,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer_args = {
    "model": model,
    "args": training_args,
    "compute_metrics": compute_metrics,
    "train_dataset": tokenized_datasets["train"],
    "eval_dataset": tokenized_datasets["validation"],
    "data_collator": data_collator,
    "tokenizer": tokenizer,
}

trainer = Trainer(**trainer_args)

trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: word_ids, pos, words, lid, idx.
***** Running training *****
  Num examples = 27893
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4360


Epoch,Training Loss,Validation Loss,Precision,Recall,Macro F1,Micro F1,Accuracy,Art F1,Art Precision,Art Recall,Conj F1,Conj Precision,Conj Recall,Dj F1,Dj Precision,Dj Recall,Dp F1,Dp Precision,Dp Recall,Dv F1,Dv Precision,Dv Recall,Erb F1,Erb Precision,Erb Recall,Et F1,Et Precision,Et Recall,Nk F1,Nk Precision,Nk Recall,Ntj F1,Ntj Precision,Ntj Recall,Onj F1,Onj Precision,Onj Recall,Oun F1,Oun Precision,Oun Recall,Ron F1,Ron Precision,Ron Recall,Ropn F1,Ropn Precision,Ropn Recall,Um F1,Um Precision,Um Recall,Unct F1,Unct Precision,Unct Recall,Ux F1,Ux Precision,Ux Recall,F1,Precision,Recall
1,0.811300,0.107150,0.957802,0.960593,0.892303,0.959195,0.965482,0.975912,0.972565,0.979282,0.905767,0.882998,0.929740,0.870998,0.862873,0.879278,0.959092,0.973219,0.945369,0.904090,0.889794,0.918852,0.965609,0.966622,0.964598,0.968785,0.961758,0.975914,1.000000,1.000000,1.000000,0.939919,0.963719,0.917266,0.994845,0.995701,0.993991,0.943732,0.931459,0.956333,0.974858,0.975969,0.973749,0.865221,0.923541,0.813830,0.968454,0.968454,0.968454,1.000000,1.000000,1.000000,0.931870,0.909010,0.955910,0.000000,0.000000,0.000000
2,0.087400,0.096968,0.962434,0.963851,0.897846,0.963142,0.968931,0.986264,0.980874,0.991713,0.906768,0.891993,0.922040,0.879100,0.866913,0.891635,0.963253,0.965013,0.961498,0.906375,0.912281,0.900544,0.970083,0.968152,0.972021,0.971569,0.975281,0.967886,1.000000,1.000000,1.000000,0.942238,0.945652,0.938849,0.996129,0.998276,0.993991,0.951246,0.951934,0.950559,0.977479,0.977076,0.977883,0.895871,0.907273,0.884752,0.978056,0.971963,0.984227,1.000000,1.000000,1.000000,0.938945,0.926095,0.952158,0.000000,0.000000,0.000000
3,0.063500,0.096270,0.962650,0.964822,0.898763,0.963735,0.969741,0.984279,0.974290,0.994475,0.907795,0.896714,0.919153,0.889952,0.895954,0.884030,0.966075,0.969110,0.963059,0.910253,0.904692,0.915883,0.970585,0.970863,0.970308,0.971606,0.966461,0.976806,1.000000,1.000000,1.000000,0.941997,0.961769,0.923022,0.995273,0.996558,0.993991,0.952878,0.949803,0.955973,0.974814,0.973608,0.976023,0.896116,0.913444,0.879433,0.978056,0.971963,0.984227,1.000000,1.000000,1.000000,0.939297,0.914667,0.965291,0.000000,0.000000,0.000000
4,0.049900,0.104851,0.963247,0.964665,0.898674,0.963955,0.969981,0.982794,0.979424,0.986188,0.915385,0.914505,0.916266,0.883430,0.895508,0.871673,0.965104,0.966111,0.964100,0.909674,0.907435,0.911925,0.969518,0.970442,0.968595,0.971466,0.963581,0.979483,1.000000,1.000000,1.000000,0.949782,0.960972,0.938849,0.995701,0.997416,0.993991,0.953442,0.949857,0.957055,0.975952,0.974644,0.977263,0.900000,0.923507,0.877660,0.968354,0.971429,0.965300,1.000000,1.000000,1.000000,0.936847,0.908370,0.967167,0.000000,0.000000,0.000000
5,0.034000,0.108149,0.964319,0.965981,0.900564,0.965149,0.971000,0.986924,0.983539,0.990331,0.918001,0.925636,0.910491,0.885823,0.886667,0.884981,0.964406,0.963155,0.965661,0.909631,0.910307,0.908956,0.970406,0.970314,0.970499,0.971896,0.964427,0.979483,1.000000,1.000000,1.000000,0.947406,0.948773,0.946043,0.995701,0.997416,0.993991,0.956867,0.957040,0.956694,0.977897,0.977291,0.978504,0.905830,0.916515,0.895390,0.978056,0.971963,0.984227,1.000000,1.000000,1.000000,0.940744,0.921692,0.960600,0.000000,0.000000,0.000000


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: word_ids, pos, words, lid, idx.
***** Running Evaluation *****
  Num examples = 4298
  Batch size = 32
/home/jmperez/.cache/pypoetry/virtualenvs/pysentimiento-bwlKzHxB-py3.7/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: INTJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/jmperez/.cache/pypoetry/virtualenvs/pysentimiento-bwlKzHxB-py3.7/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CONJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/jmperez/.cache/pypoetry/virtualenvs/pysentimiento-bwlKzHxB-py3.7/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/jmperez/.cache/

TrainOutput(global_step=4360, training_loss=0.1496922724837557, metrics={'train_runtime': 613.114, 'train_samples_per_second': 227.47, 'train_steps_per_second': 7.111, 'total_flos': 2300710943790330.0, 'train_loss': 0.1496922724837557, 'epoch': 5.0})